In [1]:
import pyodbc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

# QUERY LINES: PICKING - PACKING - BATCHING

In [2]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

In [3]:
df_operations = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = getdate() ,@to = dateadd(day, 1, getdate());\
\
    select t.completedDateTime Date, w.name Warehouse, p.name Project,\
        oc.name OperationCode, t.employee Employee\
    from datex_footprint.Tasks t\
        left join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        left join datex_footprint.Projects p\
            on p.id = t.projectId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND\
        (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') AND\
        t.statusId = 2 AND t.operationCodeId in (8, 39, 2067)", cnxn)

In [4]:
# Adding Hours
df_operations['Hour'] = pd.to_datetime(df_operations['Date']).dt.hour
df_operations.head()

,Date,Warehouse,Project,OperationCode,Employee,Hour
0,2022-09-15 12:28:10.720,10,ISDIN Corporation,Picking,Andre Jocelin,12
1,2022-09-15 12:32:07.017,10,ISDIN Corporation,Picking,Andre Jocelin,12
2,2022-09-15 12:30:00.473,10,ISDIN Corporation,Picking,Andre Jocelin,12
3,2022-09-15 12:34:25.663,10,ISDIN Corporation,Picking,Andre Jocelin,12
4,2022-09-15 12:34:09.880,10,ISDIN Corporation,Picking,Andre Jocelin,12


In [5]:
# Activity Function By Warehouse
def activity (warehouse, operationCode, df):
    # Filter by warehouse
    df2 = df[(df['Warehouse'] == warehouse) & (df['OperationCode'] == operationCode)]
    
    # Grouped by Employee and Hour
    df2 = df2.groupby(['Employee', 'Hour']).count()[['OperationCode']]
    df2.reset_index(inplace=True)
    
    # Pivot
    df2 = df2.pivot(index='Employee', columns='Hour', values='OperationCode')
    df2 = df2.fillna(0)
    df2 = df2.rename_axis(None, axis=1)
    
    # Save into JSON
    df2 = df2.T
    file_name = f'data/W{warehouse}_{operationCode}.json'
    df2.to_json(file_name)

In [6]:
activity('10', 'Picking', df_operations)
activity('10', 'Rate And Ship', df_operations)
activity('10', 'Batch Move', df_operations)

In [7]:
# # Close connection
cnxn.close()

# QUERY ORDERS: PICKING - PACKING - BATCHING

In [8]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

In [9]:
# TODAY ORDERS
df_today = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = getdate() ,@to = dateadd(day, 1, getdate());\
\
    select distinct w.name Warehouse, oc.name OperationCode,\
        COUNt(DISTINCT (ISNULL(t.shipmentId, s.id))) Shipment\
    from datex_footprint.Tasks t\
        left join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        left join datex_footprint.PickSlips ps\
            on ps.id = t.pickSlipId\
        left join datex_footprint.Waves wa\
            on wa.id = ps.waveId\
        left join datex_footprint.shipments s\
            on s.waveId = wa.id\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND\
        (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') AND\
        t.statusId = 2 AND t.operationCodeId in (8, 39, 2067)\
        group by w.name, oc.name", cnxn)

In [10]:
# YESTEDAY ORDERS
df_yesterday = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = dateadd(day, -1, getdate()) ,@to = getdate();\
\
    select distinct w.name Warehouse, oc.name OperationCode,\
        COUNt(DISTINCT (ISNULL(t.shipmentId, s.id))) Shipment\
    from datex_footprint.Tasks t\
        left join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        left join datex_footprint.PickSlips ps\
            on ps.id = t.pickSlipId\
        left join datex_footprint.Waves wa\
            on wa.id = ps.waveId\
        left join datex_footprint.shipments s\
            on s.waveId = wa.id\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND\
        (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') AND\
        t.statusId = 2 AND t.operationCodeId in (8, 39, 2067)\
        group by w.name, oc.name", cnxn)

In [11]:
# Rename Columns
df_today.rename(columns={'Shipment': 'Today'}, inplace=True)
df_yesterday.rename(columns={'Shipment': 'Yesterday'}, inplace=True)

In [12]:
# Merge dataframes
df_today_yesterday = pd.merge(df_today, df_yesterday,  how='outer', left_on=['Warehouse','OperationCode'], right_on = ['Warehouse','OperationCode'])

In [13]:
def orders (warehouse, df):
    df2 = df[df['Warehouse'] == warehouse]
    df2 = df2.pivot(index='Warehouse', columns='OperationCode', values=['Today', 'Yesterday']).fillna(0)
    df2 = df2.T
    
    display(df2)
    
    file_name = f'data/W{warehouse}_Orders.json'
    df2.to_json(file_name)

In [14]:
orders('10', df_today_yesterday)

Warehouse                   10
          OperationCode       
Today     Batch Move     166.0
          Picking        128.0
          Rate And Ship   83.0
Yesterday Batch Move     123.0
          Picking        119.0
          Rate And Ship   94.0

In [15]:
# # Close connection
cnxn.close()

# QUERY: OPEN ORDERS
Shipments in Released and Executing status

In [16]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

In [17]:
# OPEN ORDERS
df_open_orders = pd.read_sql_query("select w.name Warehouse, count(s.lookupCode) Shipments\
    from datex_footprint.Shipments s\
        join datex_footprint.Warehouses w\
            on w.id = ISNULL(s.actualWarehouseId, s.expectedWarehouseId)\
    where s.statusId in (2) and s.typeId = 2\
    group by w.name", cnxn)

In [18]:
df_open_orders.set_index('Warehouse', inplace=True)
df_open_orders = df_open_orders.T
df_open_orders

Warehouse,10,102,15,16,17,19,20,21,22
Shipments,55,1,19,12,13,2,7,3,1


In [19]:
file_name = f'data/Open_Orders.json'
df_open_orders.to_json(file_name)

In [20]:
# # Close connection
cnxn.close()

# QUERY 02: Open Outbounds WAREHOUSE 10

In [21]:
# Connect to the Database

# cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                         "Server=wd02;"
#                         "Database=FootPrint;"
#                         "Trusted_Connection=yes;")

In [22]:
# df_02 = pd.read_sql_query("SELECT DISTINCT w.name warehouse, p.name project, s.lookupCode shipment_number, s.createdSysDateTime\
#     FROM datex_footprint.shipments s\
#     LEFT JOIN datex_footprint.ShipmentOrderLookup sol\
#         on sol.shipmentId = s.id\
#     LEFT JOIN datex_footprint.orders o\
#         on o.id = sol.orderId\
#     LEFT JOIN datex_footprint.Projects p\
#         on p.id = o.projectId\
#     LEFT JOIN datex_footprint.warehouses w\
#         on w.id = isnull(s.actualWarehouseId, s.expectedWarehouseId)\
#     WHERE s.statusId in (1, 2, 4) and s.typeId = 2\
#     ORDER BY createdSysDateTime", cnxn)

In [23]:
# df_02.head()

In [24]:
# df_02['age_order'] = pd.Timestamp.now() - df_02['createdSysDateTime']
# df_02.head()

In [25]:
# df_02['age_order_days'] = df_02['age_order'].dt.components['days']
# df_02['age_order_hours'] = df_02['age_order'].dt.components['hours']
# df_02.head()

In [26]:
# Open Orders less than 24 hours

# df_02 = df_02.loc[(df_02['age_order_days'] == 0) & (df_02['warehouse'] == '10')]
# df_02.head()

In [27]:
# Bins per hour

# bins =[-1,4,8,12,16,24]
# bin_names = ['4 Hours', '8 Hours', '12 Hours', '16 Hours', '24 Hours']

# df_02['age_group'] = pd.cut(df_02['age_order_hours'], bins, labels=bin_names)

In [28]:
# df_02_grouped = df_02.groupby(['age_group'])[['project']].count()

In [29]:
# df_02_grouped = df_02_grouped.rename(columns={'project': 'orders'})

### Save in JSON File

In [30]:
# open_outbounds = df_02_grouped
# open_outbounds.to_json('data/open_outbounds.json')
# open_outbounds

In [31]:
# df_02_grouped = df_02_grouped.iloc[::-1]
# df_02_grouped.plot.barh()

In [32]:
# Close connection

# cnxn.close()

# QUERY 3: Picking Performance (Orders and Waves)

In [33]:
# Connect to the Database

# cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                         "Server=wd02;"
#                         "Database=FootPrint;"
#                         "Trusted_Connection=yes;")

In [34]:
# df03 = pd.read_sql_query("DECLARE @from date, @to date;\
#                           SELECT @from = getdate(), @to = dateadd(day, 1, getdate());\
#     SELECT t.completedDateTime Date, oc.name Operation, w.name Warehouse,\
#            p.name Project, m.lookupCode Material, employee Employee, s.lookupCode Shipment, isnull(s.waveId, ps.waveId) Wave,\
#            ordersPerWave.shipmentsCount, lc.name Location\
#     FROM datex_footprint.Tasks t\
#         join datex_footprint.OperationCodes oc\
#             on oc.id = t.operationCodeId\
#         left join datex_footprint.Warehouses w\
#             on w.id = t.warehouseId\
#         left join datex_footprint.Projects p\
#             on p.id = t.projectId\
#         left join datex_footprint.Materials m\
#             on m.id = t.materialId\
#         left join datex_footprint.Shipments s\
#             on s.id = t.shipmentId\
#         left join datex_footprint.PickSlips ps\
#             on ps.id = t.pickSlipId\
#         left join datex_footprint.LocationContainers lc\
#             on lc.id = t.actualSourceLocationId\
#         outer apply (\
#             select\
#                 count(distinct tt.shipmentId) shipmentsCount\
#                 from datex_footprint.waves ww\
#                     inner join datex_footprint.PickSlips ppss\
#                     inner join datex_footprint.tasks tt\
#                         on tt.operationCodeId = 8 and tt.pickSlipId = ppss.id\
#                         on ppss.waveId = ww.id\
#                 where ww.id = isnull(s.waveId, ps.waveId)) ordersPerWave\
#     where t.completedDateTime >= @from and t.completedDateTime < @to and t.statusId = 2\
#     and t.operationCodeId in (8, 39, 2064, 2065, 2066) and t.employee != 'Foot Print Service'\
#     order by s.lookupCode, t.completedDateTime", cnxn)

In [35]:
# df03.tail(3)

In [36]:
# Order Columns

# df03 = df03[['Warehouse', 'Project', 'Wave', 'Shipment', 'Date', 'Operation', 'Employee',
#              'Location', 'shipmentsCount', 'Material']]

### Filter by Shipments

In [37]:
# Filter by Orders only

# df03_orders = df03.loc[(df03['Operation']=='Picking') |
#                        (df03['Operation']=='Scanned Shipment') |
#                        (df03['Operation']=='Picking Completed')]

# df03_orders = df03_orders[['Warehouse', 'Project', 'Shipment', 'Date', 'Operation', 'Employee', 'Location']]

# df03_orders.head()

In [38]:
# Create a new column with the time difference between rows

# df03_orders['PickingTime'] = df03_orders['Date'].diff()
# df03_orders.head()

In [39]:
# Add Hour, Minute and Second columns

# df03_orders['hour'] = df03_orders['PickingTime'].dt.components['hours']
# df03_orders['minute'] = df03_orders['PickingTime'].dt.components['minutes']
# df03_orders['second'] = df03_orders['PickingTime'].dt.components['seconds']
# df03_orders.head()

In [40]:
# Set all the Scanned Shipment hour, minute, time to ZERO

# df03_orders_modified = df03_orders

# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'hour'] = 0.0
# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'minute'] = 0.0
# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'second'] = 0.0

# df03_orders_modified.loc[df03_orders_modified['Operation'] == 'Scanned Shipment'].head()

In [41]:
# df03_orders_modified['Time'] = df03_orders_modified['hour']*60 + df03_orders_modified['minute'] + df03_orders_modified['second']/60
# df03_orders_modified.head()

In [42]:
# Save in JSON Format for HTML

# df03_group = df03_orders_modified.groupby(['Warehouse', 'Project', 'Shipment', 'Operation', 'Date'])[['Time']].sum().sort_values(['Warehouse', 'Project', 'Shipment', 'Date'])
# df03_group.to_json('data/performance.json', orient='table')
# df03_group.head(20)

In [43]:
# Remove Operation 'Scanned Shipment'

# df03_excluding_Scanned = df03_orders_modified.loc[df03_orders_modified['Operation'] != 'Scanned Shipment']
# df03_excluding_Scanned.head()

In [44]:
# Save in CSV Format for Tableau

# df03_group2 = df03_excluding_Scanned.groupby(['Warehouse', 'Project', 'Employee', 'Shipment', 'Operation', 'Date'])[['Time']].sum().sort_values(['Warehouse', 'Project', 'Employee', 'Shipment', 'Date'])
# df03_group2.to_csv('data/performance.csv')
# df03_group2.head(20)

In [45]:
# Close connection

# cnxn.close()